In [1]:
from dataclasses import dataclass
import numpy as np
from numpy import exp, log, sqrt
from scipy.stats import norm

In [2]:
@dataclass
class Vasicek:
    kappa: float
    theta: float
    sigma: float
    r0: float

In [3]:
def vasicek_AB(params: Vasicek, t: float, T: float):
    """
    Bond
    """
    k, th, s = params.kappa, params.theta, params.sigma
    tau = max(T - t, 0)
    if tau == 0:
        return 1, 0
    B = (1 - np.exp(-k * tau)) / k
    A = np.exp((th - s*s/(2*k*k))*(B - tau) - (s*s)*(B*B)/(4*k))
    return A, B

def P0T(params: Vasicek, T: float) -> float:
    A, B = vasicek_AB(params, 0.0, T)
    return A * np.exp(-B * params.r0)

In [4]:
def vasicek_bond_option_0(params: Vasicek, T: float, S: float, K: float, call: bool = True) -> float:
    """
    Price at t=0 of bond option
    """
    P0S = P0T(params, S)
    P0T_ = P0T(params, T)

    k, s = params.kappa, params.sigma
    sigP2 = (s*s / (2 * k**3)) * (1 - np.exp(-k*T))**2 * (1 - np.exp(-2*k*(S - T)))
    sigP = np.sqrt(max(sigP2, 0))
    x = np.log(P0S / (K * P0T_))
    d1 = (x + 0.5 * sigP*sigP) / sigP
    d2 = d1 - sigP

    if call:
        return P0S * norm.cdf(d1) - K * P0T_ * norm.cdf(d2)
    else:
        return K * P0T_ * norm.cdf(-d2) - P0S * norm.cdf(-d1)

In [6]:
def jamshidian_swaption_vasicek(
    params: Vasicek,
    T_exp: float,
    pay_times: np.ndarray, 
    deltas: np.ndarray,  
    K: float,    
    notional: float = 1.0,
    payer: bool = True,  
):
    """
    Jamshidian trick
    """
    #assert np.all(pay_times > T_exp), "All cashflows must be after option expiry."
    N = len(pay_times)
    c = K * deltas.copy()
    c[-1] += 1  # notional for the last payment

    def P_TU_at_r(TU, r):
        A, B = vasicek_AB(params, T_exp, TU)
        return A * np.exp(-B * r)
    def g_of_r(r): # solver
        return P_TU_at_r(pay_times[-1], r) + K * np.sum(deltas * np.array([P_TU_at_r(ti, r) for ti in pay_times])) - 1.0

    rL, rH = -0.2, 1
    for _ in range(20):
        gL, gH = g_of_r(rL), g_of_r(rH)
        if gL * gH <= 0:
            break
        rL -= 1/4
        rH += 1/4

    for _ in range(100):
        rm = 0.5 * (rL + rH)
        gm = g_of_r(rm)
        if abs(gm) < 1e-14:
            r_star = rm
            break
        if gL * gm <= 0:
            rH, gH = rm, gm
        else:
            rL, gL = rm, gm
    else:
        r_star = 0.5 * (rL + rH)

    K_strikes = np.array([P_TU_at_r(ti, r_star) for ti in pay_times])

    # sum of ZC bond options
    price = 0
    for j, (tj, cj, Kj) in enumerate(zip(pay_times, c, K_strikes)):
        if cj == 0:
            continue
        if payer:
            price += cj * vasicek_bond_option_0(params, T_exp, tj, Kj, call=False)
        else:
            price += cj * vasicek_bond_option_0(params, T_exp, tj, Kj, call=True)

    return notional * price, r_star, K_strikes

In [7]:
def forward_swap_rate_and_pvbp(params: Vasicek, T_exp: float, pay_times: np.ndarray, deltas: np.ndarray):
    A0 = np.sum(deltas * np.array([P0T(params, ti) for ti in pay_times]))
    S0 = (P0T(params, T_exp) - P0T(params, pay_times[-1])) / A0
    return S0, A0

vas = Vasicek(
    kappa=0.6,
    theta=0.03,
    sigma=0.02,
    r0=0.02
)
T_exp = 2
tenor_years = 3
freq = 2  # semi-annual
pay_times = T_exp + np.arange(1, int(tenor_years * freq) + 1) / freq
deltas = np.full_like(pay_times, 1/freq, dtype=float)
S0, A0 = forward_swap_rate_and_pvbp(vas, T_exp, pay_times, deltas)
K = S0 + 0.0025
payer_px, r_star_pay, Kjs = jamshidian_swaption_vasicek(vas, T_exp, pay_times, deltas, K, payer=True)
recv_px,  r_star_rec, _    = jamshidian_swaption_vasicek(vas, T_exp, pay_times, deltas, K, payer=False)

print(f"Payer swaption (Jamshidian):  {payer_px:.10f}")
print(f"Receiver swaption (Jamshidian): {recv_px:.10f}")
print(f"r* (payer) = {r_star_pay:.6f},  r* (receiver) = {r_star_rec:.6f}")

Payer swaption (Jamshidian):  0.0049897648
Receiver swaption (Jamshidian): 0.0117989747
r* (payer) = 0.031757,  r* (receiver) = 0.031757


In [ ]:
# r* turns out to be the same for payer/receiver
# it is defined only by the strike K and the bond cashflows, not by option type
# once you find the r* that makes the fixed-leg bond worth par at expiry,
# that rate works for both pay and receive sides
# what a beautiful example of no arbitrage 

In [8]:
# parity check
lhs = payer_px - recv_px
rhs = A0 * (S0 - K)
print(f"Parity error: {lhs - rhs:.3e}")

Parity error: 1.003e-15
